In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy.stats as stats
import seaborn as sns

import os
import sys
sys.path.append(os.getcwd() + '/src')
import data_cleaning_eda as dce # my defined functions

%matplotlib inline
plt.style.use('ggplot')

ModuleNotFoundError: No module named 'data_cleaning_eda'

# Data Wrangling

## Loading Data to a Pandas DataFrame

In [ ]:
# Load data to dataframe
data_file = 'data/cscpopendata.csv'
data_df = pd.read_csv(data_file)

# Get dataframe shape
data_df.shape

In [ ]:
# Preview top of dataframe
data_df.head()

In [ ]:
# Preview bottom of dataframe
data_df.tail()

In [ ]:
# Get column info
data_df.info()

## Handling Null Values

In [ ]:
# Show cols with null vals & how many they have
print('Columns with Null Values:')
null_count = data_df.isna().sum()
null_count_df = null_count[null_count > 0].to_frame().reset_index().sort_values(0, ascending=False)
null_count_df.rename(columns={'index': 'Column', 0: 'Number of Null Values'})

In [ ]:
# Replace all null BrandName values with 'Missing'
data_df2 = data_df.copy()
data_df2.loc[data_df2['BrandName'].isna()==True, 'BrandName'] = 'Missing'

In [ ]:
# Updated - Show updated cols with null vals & how many they have
print('Remaining columns with Null Values:')
null_count2 = data_df2.isna().sum()
null_count2_df = null_count2[null_count2 > 0].to_frame().reset_index().sort_values(0, ascending=False)
null_count2_df.rename(columns={'index': 'Column', 0: 'Number of Null Values'})

### Remaining Columns with Null Values
* ChemicalDateRemoved 
    * **KEEP**, non-null for **reformulated products** only
    * If applicable, date the chemical was removed from product, i.e., reformulation.
* DiscontinuedDate
    * **KEEP**, non-null for **discontinued products** only
    * If applicable, date the product was discontinued.
* CSF 
    * *DROP COLUMN*
    * Color, scent and/or flavor as entered by manufacturer, packer and/or distributor. The color/scent/flavor name is entered as free-text into the reporting database and may contain misspellings or other errors. **Not all products have specific colors, scents or flavors.**
* CSFId
    * *DROP COLUMN*
    * CDPH internal identification number for a color/scent/flavor. **Like CSF, not all products have CSFId.**
* CasNumber
    * *DROP COLUMN* 
    * **Refer to 'CasId'/'ChemicalName' columns instead**
    * Chemical identification number (Chemical Abstracts Service registry number assigned to specific chemical). Note that different forms of a chemical  may have different CAS numbers. What appears here is the CAS number selected by the manufacturer, packer and/or distributor. 

In [ ]:
# Check if can indeed refer to 'CasId'/'ChemicalName' columns instead of CasNumber
data_df2[data_df2['CasNumber'].isna()==True][['CasId', 'ChemicalId', 'ChemicalName']]

In [ ]:
# Drop 4 unnecessary columns
drop_cols = ['CSF', 'CSFId', 'CasNumber', 'ChemicalId']
data_df3 = data_df2.copy()
data_df3 = data_df2.drop(columns=drop_cols)

In [ ]:
# Updated - Show final cols with null vals & how many they have
print('Final columns with Null Values:')
null_count3 = data_df3.isna().sum()
null_count3_df = null_count3[null_count3 > 0].to_frame().reset_index().sort_values(0, ascending=False)
null_count3_df.rename(columns={'index': 'Column', 0: 'Number of Null Values'})

## Converting Date Str Columns to Date-Time Type

In [ ]:
date_cols = ['InitialDateReported', 'MostRecentDateReported', 'DiscontinuedDate', 
             'ChemicalCreatedAt', 'ChemicalUpdatedAt', 'ChemicalDateRemoved']
data_df4 = data_df3.copy()

data_df4[date_cols] = data_df4[date_cols].apply(pd.to_datetime)

data_df4.info()
data_df4.head()

## Initial Check for Duplicate Rows

In [ ]:
# Find number of duplicate rows
duplicates_df = data_df4[data_df4.duplicated()]
duplicates_df.shape

In [ ]:
# Drop duplicate rows
data_df5 = data_df4.copy()
data_df5 = data_df5.drop_duplicates()
data_df5.shape

## Counting Unique Values per Column

In [ ]:
# Show number of unique vals per column
print('Number of Unique Values per Column:')
num_unique_df = data_df5.nunique(axis=0).to_frame().reset_index()
num_unique_df.sort_values(0, ascending=False).rename(columns={'index': 'Column', 0: 'Number of Unique Values'})

## Handling Categorical Data

* 36,794 unique products identified by CDPHId, *IGNORE ProductName*
    * Number of unique CDPHId vals > Number of unique ProductName vals
    * Can have multiple categories (PrimaryCategory, SubCategory) 
    * Can have multiple chemicals (ChemicalName)
* 123 unique chemicals identified by ChemicalName, *IGNORE CasId*
    * Number of unique CasId vals > Number of unique ChemicalName vals
* Need to investigate:
    * **Number of unique CompanyId vals > Number of unique CompanyName vals**

#### Messy Company Names

In [ ]:
# Show CompanyName counts
# df = pd.DataFrame(data_df5['CompanyName'].value_counts()).reset_index().sort_values('index')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)

In [ ]:
# Example of similar CompanyNames that could be merged - 'Fresh'
fresh_df = data_df5[data_df5['CompanyName'].str.contains('Fresh')]
fresh_df[['CompanyId','CompanyName','BrandName']].drop_duplicates()

### Replacing Categorical Columns with Cleaned Ones

In [ ]:
# Use function to clean categorical columns of interest
data_df6 = dce.clean_cat_data(data_df5, ['CompanyName', 'ProductName', 'SubCategory', 
                                         'BrandName'])
data_df6.head()

In [ ]:
# # Show Cleaned CompanyName counts - Sorted by name
# df = pd.DataFrame(data_df6['Cleaned_CompanyName'].value_counts()).reset_index().sort_values('index')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)

In [ ]:
# # Show Cleaned CompanyName counts - Sorted by count, descending
# df = pd.DataFrame(data_df6['Cleaned_CompanyName'].value_counts()).reset_index().sort_values('Cleaned_CompanyName', ascending=False)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df)

In [ ]:
# Same example of CompanyNames after cleaning - 'Fresh'
fresh_df6 = data_df6[data_df6['Cleaned_CompanyName'].str.contains('fresh')]
fresh_df6[['CompanyId','Cleaned_CompanyName','Cleaned_BrandName']].drop_duplicates()

In [ ]:
# # Updated - Show number of unique vals per column
# print('Number of Unique Values per Column:')
# num_unique_df = data_df6.nunique(axis=0).to_frame().reset_index()
# num_unique_df.sort_values(0, ascending=False).rename(columns={'index': 'Column', 0: 'Number of Unique Values'})

## Final Check for Duplicate Rows

In [ ]:
# Find number of duplicate rows
duplicates_df6 = data_df6[data_df6.duplicated()]
duplicates_df6.shape

In [ ]:
# Drop duplicate rows
# N/A

In [ ]:
# Take final df after cleaning
clean_data_df = data_df6

# EDA

* 36,794 unique products identified by CDPHId
    * Can have multiple categories (PrimaryCategory, Cleaned_SubCategory) 
    * Can have multiple chemicals (ChemicalName)
* 123 unique chemicals identified by ChemicalName

In [ ]:
# Number of unique vals per column
print('Number of Unique Values per Column:')
num_unique_df = clean_data_df.nunique(axis=0).to_frame().reset_index()
num_unique_df.sort_values(0, ascending=False).rename(columns={'index': 'Column', 0: 'Number of Unique Values'})

In [ ]:
clean_data_df.info()

### Classifying Products by Status

In [ ]:
# Modify df to add Reformulated/Discontinued status
# TODO: Make function
clean_data_df2 = clean_data_df.copy()
clean_data_df2['Count'] = 1
clean_data_df2['Reformulated'] = 0
clean_data_df2['Discontinued'] = 0

clean_data_df2.loc[(clean_data_df2['DiscontinuedDate'].isna() == False), 'Discontinued'] = 1
clean_data_df2.loc[(clean_data_df2['ChemicalDateRemoved'].isna() == False), 'Reformulated'] = 1

clean_data_df2['Only Reformulated'] = (clean_data_df2['Discontinued'] != 1) & (clean_data_df2['Reformulated'] == 1)
clean_data_df2['Only Reformulated'] = clean_data_df2['Only Reformulated'].astype(int)

clean_data_df2['Only Discontinued'] = (clean_data_df2['Discontinued'] == 1) & (clean_data_df2['Reformulated'] != 1)
clean_data_df2['Only Discontinued'] = clean_data_df2['Only Discontinued'].astype(int)

clean_data_df2['Reformulated or Discontinued'] = clean_data_df2['Discontinued'] | clean_data_df2['Reformulated']
clean_data_df2['Reformulated and then Discontinued'] = clean_data_df2['Discontinued'] & clean_data_df2['Reformulated']
clean_data_df2['Not yet Reformulated or Discontinued'] = (clean_data_df2['Discontinued'] != 1) & (clean_data_df2['Reformulated'] != 1)
clean_data_df2['Not yet Reformulated or Discontinued'] = clean_data_df2['Not yet Reformulated or Discontinued'].astype(int)

In [ ]:
clean_data_df2.head()

In [ ]:
clean_data_df2_for_status_pie = clean_data_df2.copy()
clean_data_df2_for_status_pie = pd.concat([clean_data_df2_for_status_pie.loc[:, 'CDPHId'], 
                                   clean_data_df2_for_status_pie.loc[:, 'Count':]], axis=1)
clean_data_df2_for_status_pie.drop_duplicates(inplace=True)
clean_data_df2_for_status_pie.drop(columns=['CDPHId'], inplace=True)
clean_data_df2_for_status_pie.head()

clean_data_df2_for_status_pie = clean_data_df2_for_status_pie.apply(lambda x: np.sum(x))
clean_data_df2_for_status_pie

In [ ]:
cols = ['Only Reformulated', 'Only Discontinued', 'Reformulated and then Discontinued', 'Not yet Reformulated or Discontinued']
clean_data_df2_for_status_bar = clean_data_df2_for_status_pie.loc[cols].copy()

clean_data_df2_for_status_bar = pd.DataFrame(clean_data_df2_for_status_bar, columns=['Count'])
clean_data_df2_for_status_bar.plot.barh()

In [ ]:
# Plot pie chart for product status
clean_data_df2_for_status_pie2 = clean_data_df2_for_status_bar / 37478
ax = clean_data_df2_for_status_pie2.plot.pie(y='Count', autopct='%1.1f%%', figsize=(10, 8), 
                                            textprops={'fontsize': 14, 'color': 'black', 'weight':'bold'},
                                            pctdistance=1.1, labels=None)
ax.set_ylabel('')
ax.set_title(label='Breakdown on Status of Products\n(36,794 Total)',
             size=16, color='black', weight='bold')
ax.legend(labels=clean_data_df2_for_status_pie2.index, loc='lower right', fontsize=14)
plt.tight_layout()
plt.savefig('images/pie_product_status')

In [ ]:
clean_data_df2_for_status_pie['% Reformulated Only'] = clean_data_df2_for_status_pie['Only Reformulated'] / clean_data_df2_for_status_pie['Count']
clean_data_df2_for_status_pie['% Discontinued Only'] = clean_data_df2_for_status_pie['Only Discontinued'] / clean_data_df2_for_status_pie['Count']
clean_data_df2_for_status_pie['% Reformulated and then Discontinued'] = clean_data_df2_for_status_pie['Reformulated and then Discontinued'] / clean_data_df2_for_status_pie['Count']
clean_data_df2_for_status_pie['% Not yet Reformulated or Discontinued'] = clean_data_df2_for_status_pie['Not yet Reformulated or Discontinued'] / clean_data_df2_for_status_pie['Count']
clean_data_df2_for_status_pie

# Analyzing Top Chemicals

In [ ]:
chemicals_count_df = dce.get_count_by_mult_df_sorted_count(clean_data_df2, 'CDPHId', ['ChemicalName'])
chemicals_count_df.head(15)
top_10_chemicals = list(chemicals_count_df.head(10)['ChemicalName'])
chemicals_top10_df = clean_data_df2[clean_data_df2['ChemicalName'].isin(top_10_chemicals)]

In [ ]:
chemicals_top10_df['ChemicalName'].unique()

In [ ]:
chemicals_top10_count_df = chemicals_top10_df.loc[:, ['CDPHId', 'ChemicalName', 'Only Reformulated', 'Only Discontinued', 'Reformulated and then Discontinued', 'Not yet Reformulated or Discontinued']].drop_duplicates().copy()
chemicals_top10_count_df.drop(columns=['CDPHId'], inplace=True)

chemicals_top10_count_df = chemicals_top10_count_df.groupby('ChemicalName').sum()
chemicals_top10_count_df = chemicals_top10_count_df.rename(index=
                                                           {'Retinol/retinyl esters, when in daily dosages in excess of 10,000 IU, or 3,000 retinol equivalents.':
                                                            'Retinol/retinyl esters',
                                                           'Silica, crystalline (airborne particles of respirable size)': 
                                                            'Silica, crystalline (airborne particles)'})

chemicals_top10_count_df = chemicals_top10_count_df.sort_values('Only Reformulated', ascending=True)

In [ ]:
chemicals_top10_count_df.loc['Titanium dioxide',:].head()

In [ ]:
ax = chemicals_top10_count_df.plot.barh(figsize=(12,8))
ax.legend(loc='lower right')
ax.set_title('Number of Reported Products by Chemical (Top 10)', size=16)
ax.set_xlabel('Number of Reported Products', size=14)
ax.set_ylabel('Chemical Name', size=14)
plt.tight_layout()
plt.savefig('images/bar_top10_chemicals')

In [ ]:
makeup_chemicals_df = clean_data_df2.loc[clean_data_df2['PrimaryCategory'] == 'Makeup Products (non-permanent)',['CDPHId','ChemicalName']]
makeup_chemicals_df.drop_duplicates()
makeup_chemicals_df.head()                              
makeup_chemicals_count_df = dce.get_count_by_mult_df_sorted_count(makeup_chemicals_df, 'CDPHId', ['ChemicalName'])
makeup_chemicals_count_df.head()
fig, ax = plt.subplots()
makeup_chemicals_count_df.plot.pie(ax=ax, y='CDPHId', legend=None, labels=None, title='Chemicals Reported in Makeup Products')
ax.set_ylabel('')
makeup_chemicals_count_df.loc[makeup_chemicals_count_df['ChemicalName']=='Titanium dioxide','CDPHId'] / makeup_chemicals_count_df['CDPHId'].sum()
fig.savefig('images/pie_makeup_chemicals')

In [ ]:
all_chemicals_df = clean_data_df2.loc[:,['CDPHId','ChemicalName']]
all_chemicals_df.drop_duplicates()
all_chemicals_df.head()                              
all_chemicals_df_count_df = dce.get_count_by_mult_df_sorted_count(all_chemicals_df, 'CDPHId', ['ChemicalName'])
all_chemicals_df_count_df.head()
fig, ax = plt.subplots()
all_chemicals_df_count_df.plot.pie(ax=ax, y='CDPHId', legend=None, labels=None, title='Chemicals Reported in All Products')
ax.set_ylabel('')
all_chemicals_df_count_df.loc[all_chemicals_df_count_df['ChemicalName']=='Titanium dioxide','CDPHId'] / all_chemicals_df_count_df['CDPHId'].sum()
fig.savefig('images/pie_all_chemicals')

## Analyzing by DiscontinuedDate

In [ ]:
# Count number of products by DiscontinuedDate Month
discontinued_date_count_df = dce.get_count_by_df(clean_data_df, 'CDPHId', 'DiscontinuedDate')
discontinued_date_count_df = discontinued_date_count_df.groupby(pd.Grouper(freq='M')).sum()
discontinued_date_count_df.index = discontinued_date_count_df.index.map(lambda x: x.strftime('%Y-%m'))
discontinued_date_count_df.sort_values('CDPHId', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 4))
discontinued_date_count_df['CDPHId'].plot(ax=ax, linewidth=1);
ax.set_title('Number of Products Discontinued Over Time', fontsize=20)
ax.set_xlabel('Time by Month', fontsize=15)
ax.set_ylabel('Number of Products Discontinued', fontsize=15)
fig.tight_layout()
fig.savefig('images/num_products_discontinued_over_time_monthly')

### Analyzing Peak DiscontinuedDates
1. 2013-07, 369 products
2. 2016-10, 341 products

In [ ]:
# Count df of peak DiscontinuedDates - 1. 2013-07, 369 products
peak_discontinued_df1 = clean_data_df[clean_data_df['DiscontinuedDate'].astype(str).str.contains('2013-07')]
peak_discontinued_df1['Cleaned_CompanyName'].unique()

In [ ]:
# Count number of discontinued products at peak time by Company Name - 1. 2013-07, 369 products
peak_discontinued_company_count_df1 = dce.get_count_by_df(peak_discontinued_df1, 'CDPHId', 'Cleaned_CompanyName')
peak_discontinued_company_count_df1 = peak_discontinued_company_count_df1.reset_index().sort_values('CDPHId', ascending=False)

In [ ]:
# Plot number of products by Company Name
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(data=peak_discontinued_company_count_df1, x='CDPHId', y='Cleaned_CompanyName', 
                orient = 'h')
ax.set_title('Number of Products Discontinued by Company in July 2013', fontsize=20)
ax.set_xlabel('Number of Products Discontinued', fontsize=15)
ax.set_ylabel('Company Name', fontsize=15)

for datum, p in zip(peak_discontinued_company_count_df1['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/num_products_discontinued_by_company_jul2013')

In [ ]:
# Count df of peak DiscontinuedDates - 2. 2016-10, 341 products
peak_discontinued_df2 = clean_data_df[clean_data_df['DiscontinuedDate'].astype(str).str.contains('2016-10')]
peak_discontinued_df2['Cleaned_CompanyName'].unique()

In [ ]:
# Count number of discontinued products at peak time by Company Name - 2. 2016-10, 341 products
peak_discontinued_company_count_df2 = dce.get_count_by_df(peak_discontinued_df2, 'CDPHId', 'Cleaned_CompanyName')
peak_discontinued_company_count_df2 = peak_discontinued_company_count_df2.reset_index().sort_values('CDPHId', ascending=False)

In [ ]:
# Plot number of products by Company Name
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(data=peak_discontinued_company_count_df2, x='CDPHId', y='Cleaned_CompanyName', 
                orient = 'h')
ax.set_title('Number of Products Discontinued by Company in Oct 2016', fontsize=20)
ax.set_xlabel('Number of Products Discontinued', fontsize=15)
ax.set_ylabel('Company Name', fontsize=15)

for datum, p in zip(peak_discontinued_company_count_df2['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/num_products_discontinued_by_company_oct2016')

## Analyzing Discontinued Products

### Time to Discontinuation

In [ ]:
# Get df WITH ONLY NON-NULL DISCONTINUED DATES
discontinued_df = clean_data_df[clean_data_df['DiscontinuedDate'].isna() == False].copy()
discontinued_df.head()

In [ ]:
discontinued_df.groupby('Cleaned_CompanyName')

In [ ]:
company_count_all_discontinued_df = dce.get_count_by_df_sorted_count(discontinued_df, 'CDPHId', 'Cleaned_CompanyName')
company_count_all_discontinued_df.head(10)
company_count_over_200_discontinued_df = company_count_all_discontinued_df[company_count_all_discontinued_df['CDPHId'] > 200]

In [ ]:
# Plot number of products by Company Name
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(data=company_count_over_200_discontinued_df, x='CDPHId', y='Cleaned_CompanyName', 
                 orient = 'h')
ax.set_title('Overall Number of Products Discontinued by Company (Top 5)', fontsize=20)
ax.set_xlabel('Number of Products Discontinued ', fontsize=15)
ax.set_ylabel('Company Name', fontsize=15)

for datum, p in zip(company_count_over_200_discontinued_df['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/num_products_discontinued_by_company_barchart')

In [ ]:
discontinued_df.shape
# Drop duplicate rows before plotting
discontinued_drop_dupes_df = discontinued_df.drop_duplicates(['CDPHId', 'InitialDateReported', 'DiscontinuedDate']).copy()
discontinued_drop_dupes_df.shape

In [ ]:
discontinued_drop_dupes_df['Days_to_Discontinued'] = (discontinued_drop_dupes_df['DiscontinuedDate'] - discontinued_drop_dupes_df['InitialDateReported']).dt.days
discontinued_drop_dupes_df.head()

In [ ]:
# Only look at products discontinued after being reported
discontinued_drop_dupes_df = discontinued_drop_dupes_df[discontinued_drop_dupes_df['Days_to_Discontinued'] >= 0]

In [ ]:
binsize = int(np.sqrt(discontinued_drop_dupes_df.shape[0]))
print(binsize)
fig, ax = plt.subplots()
ax.hist(discontinued_drop_dupes_df['Days_to_Discontinued'], bins=binsize)
fig.savefig('images/hist_days_to_discontinued')
# ax.hist(discontinued_drop_dupes_df['Days_to_Discontinued'], bins=binsize)

In [ ]:
sns.set(style="whitegrid")
ax = sns.violinplot(discontinued_drop_dupes_df['Days_to_Discontinued'], orient='v')

In [ ]:
sns.set(style="whitegrid")
ax = sns.boxplot(discontinued_drop_dupes_df['Days_to_Discontinued'])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))
sns.set(style="whitegrid")
ax = sns.violinplot(ax=ax, y="PrimaryCategory", x="Days_to_Discontinued",
                    data=discontinued_drop_dupes_df, palette="muted")

ax.set_title('Distribution of Days to Product Discontinuation\nby Primary Category', size=15)
ax.set_xlabel('Number of Days', size=13)
ax.set_ylabel('Primary Category', size=13)

fig.tight_layout()
fig.savefig('images/violin_days_to_discontinued_by_cat')

In [ ]:
primary_cats = list(discontinued_drop_dupes_df['PrimaryCategory'].unique())
primary_cats_dfs = dict()
for cat in primary_cats:
    primary_cats_dfs[cat] = (discontinued_drop_dupes_df.loc[discontinued_drop_dupes_df['PrimaryCategory'] == cat, 'Days_to_Discontinued'])
    means = []
for k, v in primary_cats_dfs.items():
    means.append(np.mean(v))
    print('{} mean: {}, var: {}'.format(k, np.mean(v), np.var(v)))

sorted(means)

In [ ]:
# Skin Care Products  mean: 953.3005366726297, var: 404466.31754890695
# Hair Care Products (non-coloring) mean: 954.4873417721519, var: 492509.5409790095
primary_cats_dfs['Skin Care Products ']
primary_cats_dfs['Hair Care Products (non-coloring)']

In [ ]:
discontinued_2_companies_df = discontinued_drop_dupes_df[discontinued_drop_dupes_df['Cleaned_CompanyName'].str.contains('the procter & gamble company') | discontinued_drop_dupes_df['Cleaned_CompanyName'].str.contains('buth-na-bodhaige inc')].copy()
discontinued_2_companies_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 15))
sns.set(style="whitegrid")
ax = sns.violinplot(ax=ax, y="PrimaryCategory", x="Days_to_Discontinued",
                    hue="Cleaned_CompanyName",
                    data=discontinued_2_companies_df, palette="muted")

ax.set_title('Distribution of Days to Product Discontinuation\nby Primary Category - 2 Companies', size=15)
ax.set_xlabel('Number of Days', size=13)
ax.set_ylabel('Primary Category', size=13)
ax.legend(loc='best')

fig.tight_layout()
fig.savefig('images/violin_days_to_discontinued_by_cat_2_companies')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))
sns.set(style="whitegrid")
ax = sns.violinplot(x="Cleaned_CompanyName", y="Days_to_Discontinued",
                    data=discontinued_2_companies_df, palette="muted")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 10), sharey=True)
sns.set(style="whitegrid")
sns.violinplot(discontinued_drop_dupes_df['Days_to_Discontinued'], ax=axs[1], orient='v')
sns.violinplot(ax=axs[0], x="Cleaned_CompanyName", y="Days_to_Discontinued",
                    data=discontinued_2_companies_df, palette="muted")

axs[1].set_title('Overall \nDistribution of Days to Product Discontinuation', size=15)
axs[1].set_ylabel('Number of Days', size=13)
axs[1].set_xlabel('All Companies', size=13)

axs[0].set_title('Buth-Na-Bodhaige vs Proctor & Gamble \nDistribution of Days to Product Discontinuation', size=15)
axs[0].set_ylabel('Number of Days', size=13)
axs[0].set_xlabel('Company', size=13)

fig.savefig('images/violin_days_to_discontinued')

In [ ]:
second_discontinued_2_companies_df = discontinued_drop_dupes_df[discontinued_drop_dupes_df['Cleaned_CompanyName'].str.contains('yves rocher inc') | discontinued_drop_dupes_df['Cleaned_CompanyName'].str.contains("victoria's secret beauty") | discontinued_drop_dupes_df['Cleaned_CompanyName'].str.contains("new avon llc")].copy()
second_discontinued_2_companies_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10), sharey=True)
sns.set(style="whitegrid")
sns.violinplot(ax=ax, x="Cleaned_CompanyName", y="Days_to_Discontinued",
                    data=second_discontinued_2_companies_df, palette="muted")

# axs[1].set_title('Overall \nDistribution of Days to Product Discontinuation', size=15)
# axs[1].set_ylabel('Number of Days', size=13)
# axs[1].set_xlabel('All Companies', size=13)

ax.set_title("Comparing 3 Companies \nDistribution of Days to Product Discontinuation", size=15)
ax.set_ylabel('Number of Days', size=13)
ax.set_xlabel('Company', size=13)

fig.savefig('images/violin_days_to_discontinued_3_companies')

In [ ]:
avon_times = second_discontinued_2_companies_df.loc[second_discontinued_2_companies_df['Cleaned_CompanyName'] ==  'new avon llc','Days_to_Discontinued']
yves_times = second_discontinued_2_companies_df.loc[second_discontinued_2_companies_df['Cleaned_CompanyName'] ==  'yves rocher inc','Days_to_Discontinued']
victoria_times = second_discontinued_2_companies_df.loc[second_discontinued_2_companies_df['Cleaned_CompanyName'] ==  "victoria's secret beauty",'Days_to_Discontinued']

In [ ]:
companies = [avon_times, yves_times, victoria_times]
means = [np.mean(company) for company in companies]
var = [np.var(company) for company in companies]
means
var

# Hypothesis Testing: Comparing Time to Product Discontinuation

**Null Hypothesis:** The Proctor & Gamble Company's time to product discontinuation is less than Buth-Na-Bodhaige Inc's.

**Alternate Hypothesis:** The Proctor & Gamble Company's time to product discontinuation is greater than Buth-Na-Bodhaige Inc's.

## Non-Parametrics: Mann-Whitney U Test

In [ ]:
pg_times = discontinued_2_companies_df.loc[discontinued_2_companies_df['Cleaned_CompanyName'] ==  'the procter & gamble company','Days_to_Discontinued']
bnb_times = discontinued_2_companies_df.loc[discontinued_2_companies_df['Cleaned_CompanyName'] ==  'buth-na-bodhaige inc','Days_to_Discontinued']

print('pg_times:', np.mean(pg_times))
print('bnb_times:', np.mean(bnb_times))

print('pg_times:', np.median(pg_times))
print('bnb_times:', np.median(bnb_times))

res = stats.mannwhitneyu(pg_times, bnb_times, alternative="greater")
print(res)
print("p-value for Proctor & Gamble < Buth-Na-Bodhaige: {:2.3f}".format(res.pvalue))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 10), sharey=True)
sns.set(style="whitegrid")
sns.violinplot(discontinued_drop_dupes_df['Days_to_Discontinued'], ax=axs[0], orient='v')
sns.violinplot(ax=axs[1], x="Cleaned_CompanyName", y="Days_to_Discontinued",
                    data=discontinued_2_companies_df, palette="muted")

**Null Hypothesis:** The Proctor & Gamble Company's time to product discontinuation is less than Buth-Na-Bodhaige Inc's.

**Alternate Hypothesis:** The Proctor & Gamble Company's time to product discontinuation is greater than Buth-Na-Bodhaige Inc's.

## Non-Parametrics: Mann-Whitney U Test

In [ ]:
pg_times = discontinued_2_companies_df.loc[discontinued_2_companies_df['Cleaned_CompanyName'] ==  'the procter & gamble company','Days_to_Discontinued']
bnb_times = discontinued_2_companies_df.loc[discontinued_2_companies_df['Cleaned_CompanyName'] ==  'buth-na-bodhaige inc','Days_to_Discontinued']

print('pg_times:', np.mean(pg_times))
print('bnb_times:', np.mean(bnb_times))

print('pg_times:', np.median(pg_times))
print('bnb_times:', np.median(bnb_times))

res = stats.mannwhitneyu(pg_times, bnb_times, alternative="greater")
print(res)
print("p-value for Proctor & Gamble < Buth-Na-Bodhaige: {:2.3f}".format(res.pvalue))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 10), sharey=True)
sns.set(style="whitegrid")
sns.violinplot(discontinued_drop_dupes_df['Days_to_Discontinued'], ax=axs[0], orient='v')
sns.violinplot(ax=axs[1], x="Cleaned_CompanyName", y="Days_to_Discontinued",
                    data=discontinued_2_companies_df, palette="muted")

In [ ]:
stat, p_val = stats.ttest_ind(pg_times, bnb_times, equal_var=False)

print('The statistic is: {} \nP-value: {}'.format(stat ,p_val))

## Analyzing by InitialDateReported

In [ ]:
# Count number of products by InitialDateReported Month
init_date_count_df = dce.get_count_by_df(clean_data_df, 'CDPHId', 'InitialDateReported')
init_date_count_df = init_date_count_df.groupby(pd.Grouper(freq='M')).sum()
init_date_count_df.index = init_date_count_df.index.map(lambda x: x.strftime('%Y-%m'))
init_date_count_df

In [ ]:
fig, ax = plt.subplots(figsize=(11, 4))
init_date_count_df['CDPHId'].plot(ax=ax, linewidth=1);
ax.set_title('Number of Products Reported Over Time', fontsize=20)
ax.set_xlabel('Time by Month', fontsize=15)
ax.set_ylabel('Number of Products Reported', fontsize=15)
fig.tight_layout()
fig.savefig('images/num_products_reported_over_time_monthly')

In [ ]:
# Check for products with multiple InitialDateReported values
count_dupe_CDPHId = clean_data_df.drop_duplicates(['CDPHId','InitialDateReported'])
count_dupe_CDPHId = count_dupe_CDPHId['CDPHId'].value_counts()
mult_reported = count_dupe_CDPHId[count_dupe_CDPHId > 1]
print('Number of products with multiple InitialDateReported:', len(mult_reported))
mult_reported

In [ ]:
# Example of products with multiple InitialDateReported values
clean_data_df[clean_data_df['CDPHId']==40190].sort_values('InitialDateReported').head()

In [ ]:
clean_data_df[clean_data_df['CDPHId']==40190].sort_values('InitialDateReported').tail()

## Analyzing Cosmetics Companies

In [ ]:
# Count number of products by Company Name
company_count_df = dce.get_count_by_df(clean_data_df, 'CDPHId', 'Cleaned_CompanyName')
company_count_df = company_count_df.reset_index().sort_values('CDPHId', ascending=False)
company_count_df

company_count_over500_df = company_count_df[company_count_df['CDPHId'] > 500]

In [ ]:
clean_data_df.iloc[company_count_over500_df.index][['Cleaned_CompanyName', 'PrimaryCategory', 'Cleaned_SubCategory']].drop_duplicates()

In [ ]:
# Plot number of products by Company Name
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(data=company_count_over500_df, x='CDPHId', y='Cleaned_CompanyName', 
                 orient = 'h')
ax.set_title('Number of Products Reported by Company', fontsize=20)
ax.set_xlabel('Number of Products Reported', fontsize=15)
ax.set_ylabel('Company Name', fontsize=15)

for datum, p in zip(company_count_over500_df['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/num_products_by_company_barchart')

## Analyzing Cosmetics Companies with Makeup Products Reported

In [2]:
# Count number of products by Company with Makeup Products 
makeup_company_df = 
makeup_company_count_df = dce.get_count_by_df(makeup_company_df, 'CDPHId', 'Cleaned_CompanyName')
company_count_df = company_count_df.reset_index().sort_values('CDPHId', ascending=False)
company_count_df

company_count_over500_df = company_count_df[company_count_df['CDPHId'] > 500]

SyntaxError: invalid syntax (<ipython-input-2-7e64d42f1683>, line 2)

In [ ]:
clean_data_df.iloc[company_count_over500_df.index][['Cleaned_CompanyName', 'PrimaryCategory', 'Cleaned_SubCategory']].drop_duplicates()

In [ ]:
# Plot number of products by Company Name
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(data=company_count_over500_df, x='CDPHId', y='Cleaned_CompanyName', 
                 orient = 'h')
ax.set_title('Number of Products Reported by Company', fontsize=20)
ax.set_xlabel('Number of Products Reported', fontsize=15)
ax.set_ylabel('Company Name', fontsize=15)

for datum, p in zip(company_count_over500_df['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/num_products_by_company_barchart')

# Analyzing 2 Companies of Interest
* Companies with > 300 products discontinued in a month:
    1. buth-na-bodhaige inc
    2. the procter & gamble company

In [ ]:
buth_bodhaige_df = clean_data_df[clean_data_df['Cleaned_CompanyName'] == 'buth-na-bodhaige inc']
buth_bodhaige_df.head()

In [ ]:
# Count number of products by ChemicalCount
buth_bodhaige_chem_count_df = dce.get_count_by_df(buth_bodhaige_df, 'CDPHId', 'ChemicalCount')
buth_bodhaige_chem_count_df = buth_bodhaige_chem_count_df.reset_index().sort_values('CDPHId', ascending=False)
buth_bodhaige_chem_count_df

In [ ]:
loreal_df = clean_data_df[clean_data_df['Cleaned_CompanyName'] == "l'oreal usa"]
loreal_df.head()
# Count number of products by ChemicalCount
loreal_chem_count_df = dce.get_count_by_df(loreal_df, 'CDPHId', 'ChemicalCount')
loreal_chem_count_df = loreal_chem_count_df.reset_index().sort_values('CDPHId', ascending=False)
loreal_chem_count_df

In [ ]:
# Count number of products by Cosmetic Primary Category
loreal_primary_cat_count_df = dce.get_count_by_df_sorted_count(loreal_df, 'CDPHId', 'PrimaryCategory')
loreal_primary_cat_count_df

In [ ]:
proctor_gamble_df = clean_data_df[clean_data_df['Cleaned_CompanyName'] == 'the procter & gamble company']
proctor_gamble_df.head()

In [ ]:
# Count number of products by ChemicalCount
proctor_gamble_chem_count_df = dce.get_count_by_df(proctor_gamble_df, 'CDPHId', 'ChemicalCount')
proctor_gamble_chem_count_df = proctor_gamble_chem_count_df.reset_index().sort_values('CDPHId', ascending=False)
proctor_gamble_chem_count_df

In [ ]:
# Count number of products by Cosmetic Primary Category
proctor_gamble_primary_cat_count_df = dce.get_count_by_df_sorted_count(proctor_gamble_df, 'CDPHId', 'PrimaryCategory')
proctor_gamble_primary_cat_count_df

## Analyzing Chemical Counts

In [ ]:
# Count number of products by ChemicalCount
chemical_count_df = dce.get_count_by_df(clean_data_df, 'CDPHId', 'ChemicalCount')
chemical_count_df = chemical_count_df.reset_index().sort_values('CDPHId', ascending=False)
chemical_count_df

In [ ]:
x = chemical_count_df['ChemicalCount']
y = chemical_count_df['CDPHId']

fig, ax = plt.subplots(figsize=(12, 8))
ax = sns.barplot(x, y)
ax.set_title('Number of Products Per Chemical Count', fontsize=20)
ax.set_xlabel('Chemical Count', fontsize=15)
ax.set_ylabel('Number of Products', fontsize=15)

for x_i, y_i in zip(x, y):
    ax.annotate(y_i, (x_i, y_i), textcoords="offset points", xytext=(0,4), ha='center')
    
fig.savefig('images/num_products_per_chemical_count_barchart')

## Analyzing Cosmetics Primary Categories

In [ ]:
# Count number of products by Primary Category
primary_category_count_df = dce.get_count_by_df(clean_data_df, 'CDPHId', 'PrimaryCategory')
primary_category_count_df = primary_category_count_df.reset_index().sort_values('CDPHId', ascending=False)
primary_category_count_df

In [ ]:
# Plot number of products by Primary Category
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(data=primary_category_count_df, x='CDPHId', y='PrimaryCategory', 
                 orient = 'h')
ax.set_title('Number of Products by Primary Category', fontsize=20)
ax.set_xlabel('Number of Products Reported', fontsize=15)
ax.set_ylabel('Primary Category', fontsize=15)

for datum, p in zip(primary_category_count_df['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/num_products_by_primary_cat_barchart')

In [ ]:
company_count_top10_df = company_count_df.loc[company_count_df['CDPHId'].head(10).index,:]

# Plot number of products by Company Name & by Primary Category
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(ax=ax, data=company_count_top10_df, x='CDPHId', y='Cleaned_CompanyName', 
                 orient = 'h')
ax.set_title('Number of Reported Products by Company (Top 10)', fontsize=20)
ax.set_xlabel('Number of Reported Products', fontsize=15)
ax.set_ylabel('Company Name', fontsize=15)

for datum, p in zip(company_count_over500_df['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))
        
fig.tight_layout()
fig.savefig('images/bar_num_products_by_company_top10')

In [ ]:
company_count_top10_df = company_count_df.loc[company_count_df['CDPHId'].head(10).index,:]

# Plot number of products by Company Name & by Primary Category
fig, axs = plt.subplots(2, 1, figsize=(12, 7), sharex=True)
axs[0] = sns.barplot(ax=axs[0], data=company_count_top10_df, x='CDPHId', y='Cleaned_CompanyName', 
                 orient = 'h')
axs[0].set_title('Number of Reported Products by Company (Top 10)', fontsize=20)
axs[0].set_xlabel('Number of Reported Products', fontsize=15)
axs[0].set_ylabel('Company Name', fontsize=15)

for datum, p in zip(company_count_over500_df['CDPHId'].sort_values(ascending=False), 
                    axs[0].patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        axs[0].annotate(datum, (x, y))
        
axs[1] = sns.barplot(ax=axs[1], data=primary_category_count_df, x='CDPHId', y='PrimaryCategory', 
                 orient = 'h')
axs[1].set_title('Number of Reported Products by Primary Category', fontsize=20)
axs[1].set_xlabel('Number of Reported Products', fontsize=15)
axs[1].set_ylabel('Primary Category', fontsize=15)

for datum, p in zip(primary_category_count_df['CDPHId'].sort_values(ascending=False), 
                    axs[1].patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        axs[1].annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/bar_num_products_by_company_and_primary_cat')

## See what chemicals are in makeup products

## Analyzing Cosmetics Sub Categories

In [ ]:
# Count number of products by Sub Category
sub_category_count_df = dce.get_count_by_df(clean_data_df, 'CDPHId', 'Cleaned_SubCategory')
sub_category_count_df = sub_category_count_df.reset_index().sort_values('CDPHId', ascending=False)
sub_category_count_df

sub_category_count_over500_df = sub_category_count_df[sub_category_count_df['CDPHId'] > 500]

In [ ]:
clean_data_df.iloc[sub_category_count_over500_df.index]['PrimaryCategory'].unique()

In [ ]:
# Plot number of products by Sub Category
fig, ax = plt.subplots(figsize=(12, 7))
ax = sns.barplot(data=sub_category_count_over500_df, x='CDPHId', y='Cleaned_SubCategory', 
                 orient = 'h')
ax.set_title('Number of Products by Sub Category', fontsize=20)
ax.set_xlabel('Number of Products Reported', fontsize=15)
ax.set_ylabel('Sub Category', fontsize=15)

for datum, p in zip(sub_category_count_over500_df['CDPHId'].sort_values(ascending=False), 
                    ax.patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(datum, (x, y))

fig.tight_layout()
fig.savefig('images/num_products_by_sub_cat_barchart')

In [ ]:
company_count_top10_df = company_count_df.loc[company_count_df['CDPHId'].head(10).index,:]
sub_category_count_top10_df = sub_category_count_df.loc[sub_category_count_df['CDPHId'].head(10).index,:]

# Plot number of products by Company Name, Primary Category, & Sub Category
fig, axs = plt.subplots(3, 1, figsize=(12, 15), sharex=True)
axs[0] = sns.barplot(ax=axs[0], data=company_count_top10_df, x='CDPHId', y='Cleaned_CompanyName', 
                 orient = 'h')
axs[0].set_title('Number of Reported Products by Company (Top 10)', fontsize=20)
axs[0].set_xlabel('Number of Reported Products', fontsize=15)
axs[0].set_ylabel('Company Name', fontsize=15)

for datum, p in zip(company_count_over500_df['CDPHId'].sort_values(ascending=False), 
                    axs[0].patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        axs[0].annotate(datum, (x, y))
        
axs[1] = sns.barplot(ax=axs[1], data=primary_category_count_df, x='CDPHId', y='PrimaryCategory', 
                 orient = 'h')
axs[1].set_title('Number of Reported Products by Primary Category', fontsize=20)
axs[1].set_xlabel('Number of Reported Products', fontsize=15)
axs[1].set_ylabel('Primary Category', fontsize=15)

for datum, p in zip(primary_category_count_df['CDPHId'].sort_values(ascending=False), 
                    axs[1].patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        axs[1].annotate(datum, (x, y))
        
axs[2] = sns.barplot(ax=axs[2], data=sub_category_count_top10_df, x='CDPHId', y='Cleaned_SubCategory', 
                 orient = 'h')
axs[2].set_title('Number of Reported Products by Sub Category (Top 10)', fontsize=20)
axs[2].set_xlabel('Number of Reported Products', fontsize=15)
axs[2].set_ylabel('Sub Category', fontsize=15)

for datum, p in zip(sub_category_count_top10_df['CDPHId'].sort_values(ascending=False), 
                    axs[2].patches):
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        axs[2].annotate(datum, (x, y))        

fig.tight_layout()
fig.savefig('images/bar_num_products_by_company_and_cats')

# Appendix - Additional EDA

### Investigate Products with 0 Chemicals

In [ ]:
zero_chemicals_df = clean_data_df[clean_data_df['ChemicalCount'] == 0]
zero_chemicals_df.head()

In [ ]:
zero_chemicals_df.info()

In [ ]:
# Count number of 0-chemical products by CompanyName
zero_chemicals_count_companies_df = dce.get_count_by_df(zero_chemicals_df, 'CDPHId', 'Cleaned_CompanyName')
zero_chemicals_count_companies_df = zero_chemicals_count_companies_df.reset_index().sort_values('CDPHId', ascending=False)
zero_chemicals_count_companies_df.head(10)

### Investigate Products with 0 Chemicals

In [ ]:
five_or_more_chemicals_list = [i for i in range(5,10)]
five_or_more_chemicals_df = clean_data_df[clean_data_df['ChemicalCount']
                                          .isin(five_or_more_chemicals_list)]
five_or_more_chemicals_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.hist(zero_chemicals_df['PrimaryCategory'])
plt.xticks(rotation='45')
ax.tick_params(axis="x", labelsize=20)
ax.tick_params(axis="y", labelsize=20)
ax.set_xlabel('Cosmetics Category', size=30)
plt.show()

## Discontinued Products

In [ ]:
discontinued_df = clean_data_df[clean_data_df['DiscontinuedDate'].isna()==False]
discontinued_df.head()

In [ ]:
discontinued_df

## Reformulated Products

In [ ]:
reformulated_df = clean_data_df[clean_data_df['ChemicalDateRemoved'].isna()==False]
reformulated_df.head()

### Investigate Product with 9 Chemicals

In [ ]:
nine_chemicals_df = clean_data_df[clean_data_df['ChemicalCount'] == 9]
nine_chemicals_df

In [ ]:
list(nine_chemicals_df['ChemicalName'].unique())

## Analyzing Reformulated Products

### Time to Reformulation

In [ ]:
# Get df WITH ONLY NON-NULL DISCONTINUED DATES
reformulation_df = clean_data_df[clean_data_df['ChemicalDateRemoved'].isna() == False]
reformulation_df.head()

In [ ]:
reformulation_df['Days_to_Reformulated'] = (reformulation_df['ChemicalDateRemoved'] - reformulation_df['InitialDateReported']).dt.days
reformulation_df.head()

In [ ]:
reformulation_df['ChemicalDateRemoved'].hist()

## Plot Product Counts by All Dates

In [ ]:
clean_data_df.info()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 9), sharex=True, sharey=True)

x=pd.date_range('2003-05', '2020-04')

init_date_count_df['CDPHId'].plot(x=x, ax=axs[0], linewidth=1);
discontinued_date_count_df['CDPHId'].plot(x=x, ax=axs[1], linewidth=1);

axs[0].set_title('Number of Products Reported Over Time', fontsize=20)
axs[0].set_xlabel('Time by Month', fontsize=15)
axs[0].set_ylabel('Number of Products', fontsize=15)

axs[1].set_title('Number of Products Discontinued Over Time', fontsize=20)
axs[1].set_xlabel('Time by Month', fontsize=15)
axs[1].set_ylabel('Number of Products', fontsize=15)

fig.tight_layout()
fig.savefig('images/num_products_over_time_monthly')

In [ ]:
plt.figure(figsize=(12,5))

ax1 = discontinued_date_count_df['CDPHId'].plot(color='red', grid=True, label='Sum')
ax2 = init_date_count_df['CDPHId'].plot(color='blue', grid=True, label='Count')

ax1.legend(loc=1)
ax2.legend(loc=2)

plt.show()

In [ ]:
plt.figure(figsize=(12,5))

ax1 = init_date_count_df['CDPHId'].plot(color='blue', grid=True, label='Count')
ax2 = discontinued_date_count_df['CDPHId'].plot(color='red', grid=True, secondary_y=True, label='Sum')

ax1.legend(loc=1)
ax2.legend(loc=2)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 4))

x=pd.date_range('2003-05', '2020-04')

init_date_count_df['CDPHId'].plot(x=x, ax=ax, linewidth=1);
discontinued_date_count_df['CDPHId'].plot(x=x, ax=ax, linewidth=1);

ax.set_title('Number of Products Reported Over Time', fontsize=20)
ax.set_xlabel('Time by Month', fontsize=15)
ax.set_ylabel('Number of Products Reported', fontsize=15)
fig.tight_layout()
fig.savefig('images/num_products_over_time_monthly')

# More Hypothesis Testing

In [ ]:
company_reformulated_discontinued_rates_df = pd.concat([clean_data_df2.loc[:,['CDPHId','MostRecentDateReported','ChemicalUpdatedAt','Cleaned_CompanyName']], 
                                                       clean_data_df2.loc[:,'Count':]], axis=1).copy() 
company_reformulated_discontinued_rates_df.sort_values(['CDPHId','MostRecentDateReported','ChemicalUpdatedAt'], ascending=False, inplace=True)
company_reformulated_discontinued_rates_df.tail()
company_reformulated_discontinued_rates_df.drop_duplicates(['CDPHId'], inplace=True)
company_reformulated_discontinued_rates_df.drop(columns=['CDPHId','MostRecentDateReported','ChemicalUpdatedAt'], inplace=True)
company_reformulated_discontinued_rates_count_df = company_reformulated_discontinued_rates_df.groupby('Cleaned_CompanyName').sum()
company_reformulated_discontinued_rates_count_df.sort_values('Count', ascending=False, inplace=True)
company_reformulated_discontinued_rates_count_df.head(10)

In [ ]:
company_reformulated_discontinued_rates_count_df['Ratio Reformulated or Discontinued'] = company_reformulated_discontinued_rates_count_df['Reformulated or Discontinued'] / company_reformulated_discontinued_rates_count_df['Count']
company_reformulated_discontinued_rates_count_df['Ratio Not yet Reformulated or Discontinued'] = company_reformulated_discontinued_rates_count_df['Not yet Reformulated or Discontinued'] / company_reformulated_discontinued_rates_count_df['Count']
company_reformulated_discontinued_rates_count_df.head(10)

In [ ]:
company_reformulated_discontinued_rates_count_df_top10 = company_reformulated_discontinued_rates_count_df.loc[company_reformulated_discontinued_rates_count_df.head(10).index,:]
company_reformulated_discontinued_rates_count_df_top10 = company_reformulated_discontinued_rates_count_df_top10.sort_values('Count', ascending=True)
fig, ax = plt.subplots(figsize=(10, 6))
company_reformulated_discontinued_rates_count_df_top10.plot.barh(ax=ax, y=['Ratio Reformulated or Discontinued', 'Ratio Not yet Reformulated or Discontinued'], stacked=True)
ax.set_title('Ratios of Reported Product Status for Top 10 Companies', size = 15)
ax.set_xlabel('Ratio', size = 12)
ax.set_ylabel('Company', size = 12)
ax.legend(loc='best')
fig.tight_layout()
fig.savefig('images/stacked_bar_ratios_top10_companies')

In [ ]:
company_reformulated_discontinued_rates_count_df_top10.head(10)

In [ ]:
company_reformulated_discontinued_rates_count_df['Ratio Reformulated or Discontinued'] = company_reformulated_discontinued_rates_count_df['Reformulated or Discontinued'] / company_reformulated_discontinued_rates_count_df['Count']
company_reformulated_discontinued_rates_count_df['Ratio Not yet Reformulated or Discontinued'] = company_reformulated_discontinued_rates_count_df['Not yet Reformulated or Discontinued'] / company_reformulated_discontinued_rates_count_df['Count']
company_reformulated_discontinued_rates_count_df.sort_values('Ratio Reformulated or Discontinued', ascending=False).head(30)

In [ ]:
company_reformulated_discontinued_rates_count_df.loc[company_reformulated_discontinued_rates_count_df.index.str.contains('maybe'),:]

In [ ]:
company_reformulated_discontinued_rates_count_df.sample(20)

In [ ]:
companies = ['opi products inc', 'revlon consumer product corporation', 'chanel inc', 'estee lauder inc', 'the procter & gamble company', 'buth-na-bodhaige inc']
choose_companies_reformulated_discontinued_rates_count_df = company_reformulated_discontinued_rates_count_df.loc[company_reformulated_discontinued_rates_count_df.index.isin(companies),:]
choose_companies_reformulated_discontinued_rates_count_df.head(10)

In [ ]:
company1_k = 118
company1_n = 1458
company2_k = 13
company2_n = 236

company1_ratio = company1_k / company1_n
company2_ratio = company2_k / company2_n

In [ ]:
print(company1_ratio, company2_ratio)

shared_sample_freq = (company1_k + company2_k) / (company1_n + company2_n)
shared_sample_variance = (company1_n + company2_n) * (shared_sample_freq * (1 - shared_sample_freq)) / (company1_n * company2_n)

difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))

fig, ax = plt.subplots(1, figsize=(16, 3))

x = np.linspace(-1, 1, num=250)
ax.plot(x, difference_in_proportions.pdf(x), linewidth=3)
ax.set_xlim(-1, 1)
ax.set_title("Distribution of Difference in Sample Frequencies Assuming $H_0$")

In [ ]:
company1_sample_freq = company1_k / company1_n
company2_sample_freq = company2_k / company2_n
difference_in_sample_proportions = company1_sample_freq - company2_sample_freq
print("Difference in sample proportions: {:2.2f}".format(difference_in_sample_proportions))

p_value = 1 - difference_in_proportions.cdf(difference_in_sample_proportions)
print("p-value for kickfip frequency comparison: {:2.2f}".format(p_value))

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

x = np.linspace(-1, 1, num=250)
ax.plot(x, difference_in_proportions.pdf(x), linewidth=3)
ax.fill_between(x, difference_in_proportions.pdf(x), where=(x >= difference_in_sample_proportions),
                color="red", alpha=0.5)
ax.set_xlim(-1, 1)
ax.set_title("p-value Region")

fig.savefig('images/hyp_test_opi_revlon')

In [ ]:
company1_k = 5.0
company1_n = 88
company2_k = 0
company2_n = 68

company1_ratio = company1_k / company1_n
company2_ratio = company2_k / company2_n

In [ ]:
print(company1_ratio, company2_ratio)

shared_sample_freq = (company1_k + company2_k) / (company1_n + company2_n)
shared_sample_variance = (company1_n + company2_n) * (shared_sample_freq * (1 - shared_sample_freq)) / (company1_n * company2_n)

difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))

fig, ax = plt.subplots(1, figsize=(16, 3))

x = np.linspace(-1, 1, num=250)
ax.plot(x, difference_in_proportions.pdf(x), linewidth=3)
ax.set_xlim(-1, 1)
ax.set_title("Distribution of Difference in Sample Frequencies Assuming $H_0$")

In [ ]:
company1_sample_freq = company1_k / company1_n
company2_sample_freq = company2_k / company2_n
difference_in_sample_proportions = company1_sample_freq - company2_sample_freq
print("Difference in sample proportions: {:2.2f}".format(difference_in_sample_proportions))

p_value = 1 - difference_in_proportions.cdf(difference_in_sample_proportions)
print("p-value for kickfip frequency comparison: {:2.2f}".format(p_value))

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 3))

x = np.linspace(-1, 1, num=250)
ax.plot(x, difference_in_proportions.pdf(x), linewidth=3)
ax.fill_between(x, difference_in_proportions.pdf(x), where=(x >= difference_in_sample_proportions),
                color="red", alpha=0.5)
ax.set_xlim(-1, 1)
ax.set_title("p-value Region")

fig.savefig('images/hyp_test_estee_chanel')